# Introduction to LLMs and Agents

Welcome to our workshop! In this session, we'll explore how to build AI-powered applications using **LangChain**, a popular framework for developing applications with Large Language Models (LLMs). We'll start with a simple chatbot and then enhance it with a multi-agent framework.

## Setting Up Our Environment

First, we need to set up our environment. We'll use OpenAI's models, so we need an API key. You can define your `OPENAI_API_KEY` in the `.env` file.

The code retrieve the key and sets some global configurations:
- `LLM_MODEL`: The specific model we'll use
- `LLM_TEMPERATURE`: Controls randomness in responses (0 means very deterministic)

In [ ]:
import os

In [ ]:
if not os.environ.get("OPENAI_API_KEY"):
    raise ValueError("Please set OPENAI_API_KEY environment variable")

LLM_MODEL = "gpt-4o-mini"
LLM_TEMPERATURE = 0.9

## Building a Simple ChatBot

Let's start with creating a basic chatbot using **LangChain**. We'll use:
- `ChatOpenAI`: The interface to OpenAI's chat models
- `SystemMessage`: Defines the bot's behavior and role
- `HumanMessage`: Represents user input

Our chatbot will act as a Financial Analyst. We'll create it by:
1. Instantiating the model
2. Defining a system prompt that sets the bot's role
3. Sending a user query and getting a response with `.invoke()`

This demonstrates the basic pattern of LLM interactions: prompt → response.

In [ ]:
from IPython.display import Markdown
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI

In [ ]:
# TODO: Create a ChatOpenAI instance with the LLM model and temperature
base_model = ChatOpenAI(model=LLM_MODEL, temperature=LLM_TEMPERATURE)

In [ ]:
BASE_PROMPT = """
You are a Financial Analyst. Do your best to help the client with their request based on your expertise. Give a succinct and clear response.
"""

In [ ]:
# Request from the client
request = "I want to invest in the technology sector. Can you please define an investment strategy?"

# Message list for the base model
messages = [
    SystemMessage(BASE_PROMPT),
    HumanMessage(request),
]

# Invoke the model with the messages
response = base_model.invoke(messages)

In [31]:
Markdown(response.content)

Certainly! Here’s a concise strategy for investing in the technology sector:

1. **Research and Identify Trends**: Stay updated on emerging technologies (AI, cybersecurity, cloud computing, etc.) and identify companies that are poised for growth in these areas.

2. **Diverse Portfolio**: Invest in a mix of large-cap tech stocks (established companies), mid-cap (growing firms), and small-cap (high-risk, high-reward startups) to balance potential risks and rewards.

3. **Invest in ETFs and Mutual Funds**: Consider technology-focused ETFs or mutual funds for diversification. This approach allows you to spread your investment across multiple companies with less risk than investing in individual stocks.

4. **Evaluate Fundamentals**: Analyze companies based on key metrics such as revenue growth, profit margins, R&D expenditure, and market share. Favor companies with a strong financial position and growth trajectory.

5. **Monitor Market Conditions**: Keep an eye on economic indicators and market sentiment, as tech stocks can be sensitive to interest rates and economic cycles.

6. **Long-Term Perspective**: Technology can be volatile; hence, adopt a long-term investment horizon to ride out market fluctuations.

7. **Stay Informed**: Continuously educate yourself about the sector, including potential regulatory changes, technological advancements, and competitive landscapes.

8. **Consider Risk Management**: Set stop-loss orders or allocate only a portion of your portfolio to tech investments to mitigate risk.

By combining these elements, you can create a balanced and informed approach to investing in the technology sector. Always consider consulting with a financial advisor to tailor this strategy to your specific financial situation and goals.

In [ ]:
from langchain.schema import SystemMessage, HumanMessage, AIMessage
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

# Load LLM
llm = ChatOpenAI(model=LLM_MODEL, temperature=LLM_TEMPERATURE)

# Agent Prompts
client_prompt = PromptTemplate.from_template(
    "You are a Client Interface agent. A user has asked the following question:\n{user_query}\n"
    "Rephrase it clearly for a financial analyst."
)
analyst_prompt = PromptTemplate.from_template(
    "You are a Financial Analyst. The client asked: {client_rephrased_query}\n"
    "Give clear investment advice. Do not assess risk."
)
risk_prompt = PromptTemplate.from_template(
    "You are a Risk Advisor. The analyst has recommended:\n{analyst_advice}\n"
    "Evaluate this advice from a risk perspective. Offer any cautions and ways to reduce risk."
)

# Define Chains
client_chain = LLMChain(llm=llm, prompt=client_prompt)
analyst_chain = LLMChain(llm=llm, prompt=analyst_prompt)
risk_chain = LLMChain(llm=llm, prompt=risk_prompt)

# Simulate user question
user_query = "I'm 35 and have $50,000 saved. Should I invest in real estate or index funds?"

# Step 1: Client Interface processes input
client_rephrased_query = client_chain.run(user_query)
print("\n🤖 Client Interface:\n", client_rephrased_query)

# Step 2: Financial Analyst responds
analyst_advice = analyst_chain.run(client_rephrased_query)
print("\n📈 Financial Analyst:\n", analyst_advice)

# Step 3: Risk Advisor evaluates
risk_evaluation = risk_chain.run(analyst_advice=analyst_advice)
print("\n⚠️ Risk Advisor:\n", risk_evaluation)


🤖 Client Interface:
 The user is seeking advice on whether to invest $1,000 in Microsoft or Tesla in the stock market.

📈 Financial Analyst:
 When considering an investment in Microsoft (MSFT) versus Tesla (TSLA) with a focus solely on potential upside, here are a few points to consider for each company:

### Microsoft (MSFT)

1. **Strong Financial Performance**: Microsoft has consistently shown strong revenue growth, profitability, and a solid earnings track record. Its cloud services (Azure) are a significant growth driver.

2. **Diversified Business Model**: Microsoft has a diversified portfolio that includes software (Office, Windows), cloud computing (Azure), gaming (Xbox), and LinkedIn. This diversification helps mitigate risks associated with reliance on a single segment.

3. **Stable Dividends**: Microsoft has a history of returning capital to shareholders through dividends, which can provide a steady income stream in addition to price appreciation.

4. **Market Position**: Mi